In [67]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import dask.dataframe as dd
import dask.array as da
import time

In [76]:
#处理日度3_factors
factors = pd.read_csv('daily_factors.csv')
factors = factors.drop(labels = 'datee', axis=1)
factors

,Mkt-RF,SMB,HML,RF,Date
0,0.59,-0.31,-0.21,0.019,1996-01-02
1,-0.08,-0.26,0.50,0.019,1996-01-03
2,-0.82,-0.38,0.22,0.019,1996-01-04
3,-0.15,0.69,-0.12,0.019,1996-01-05
4,0.24,-0.20,-0.11,0.019,1996-01-08
...,...,...,...,...,...
6937,0.25,-0.04,-0.79,0.022,2023-07-25
6938,0.02,0.68,1.03,0.022,2023-07-26
6939,-0.74,-0.90,0.27,0.022,2023-07-27
6940,1.14,0.53,-0.33,0.022,2023-07-28


In [77]:
daily_stock = pd.read_csv('daily_stock.csv')

C:\Users\23036531R\AppData\Local\Temp\ipykernel_3748\635042571.py:1: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  daily_stock = pd.read_csv('daily_stock.csv')


In [78]:
daily_stock = pd.DataFrame(daily_stock)
daily_stock

,PERMNO,date,COMNAM,DLSTCD,DLRET,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,NUMTRD
0,10001,1996-01-02,ENERGY WEST INC,NaN,NaN,8.75000,9.50000,-9.12500,0.0,-0.026667,8.75000,9.50000,2281.0,0.0
1,10001,1996-01-03,ENERGY WEST INC,NaN,NaN,9.50000,9.50000,9.50000,119.0,0.041096,8.75000,9.50000,2281.0,1.0
2,10001,1996-01-04,ENERGY WEST INC,NaN,NaN,8.75000,9.50000,-9.12500,0.0,-0.039474,8.75000,9.50000,2281.0,0.0
3,10001,1996-01-05,ENERGY WEST INC,NaN,NaN,8.75000,8.75000,8.75000,1384.0,-0.041096,8.75000,9.50000,2281.0,2.0
4,10001,1996-01-08,ENERGY WEST INC,NaN,NaN,8.75000,9.50000,8.75000,1000.0,0.000000,8.75000,9.50000,2281.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52770748,93436,2023-06-26,TESLA INC,NaN,NaN,240.70000,258.37000,241.05000,179758166.0,-0.060600,241.02000,241.05000,3169504.0,1940000.0
52770749,93436,2023-06-27,TESLA INC,NaN,NaN,240.85001,250.38989,250.21001,164827712.0,0.038000,250.20000,250.21001,3169504.0,1630000.0
52770750,93436,2023-06-28,TESLA INC,NaN,NaN,248.89000,259.88000,256.23999,159593987.0,0.024100,256.39001,256.41000,3169504.0,1550000.0
52770751,93436,2023-06-29,TESLA INC,NaN,NaN,253.61000,260.73999,257.50000,131177205.0,0.004917,257.51999,257.54001,3169504.0,1320000.0


In [79]:
def is_numeric(value):
    try:
        float(value)
        return True
    except:
        return False

In [80]:
daily_stock01 = daily_stock[daily_stock["RET"].map(is_numeric)]

In [81]:
daily_stock01

,PERMNO,date,COMNAM,DLSTCD,DLRET,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,NUMTRD
0,10001,1996-01-02,ENERGY WEST INC,NaN,NaN,8.75000,9.50000,-9.12500,0.0,-0.026667,8.75000,9.50000,2281.0,0.0
1,10001,1996-01-03,ENERGY WEST INC,NaN,NaN,9.50000,9.50000,9.50000,119.0,0.041096,8.75000,9.50000,2281.0,1.0
2,10001,1996-01-04,ENERGY WEST INC,NaN,NaN,8.75000,9.50000,-9.12500,0.0,-0.039474,8.75000,9.50000,2281.0,0.0
3,10001,1996-01-05,ENERGY WEST INC,NaN,NaN,8.75000,8.75000,8.75000,1384.0,-0.041096,8.75000,9.50000,2281.0,2.0
4,10001,1996-01-08,ENERGY WEST INC,NaN,NaN,8.75000,9.50000,8.75000,1000.0,0.000000,8.75000,9.50000,2281.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52770748,93436,2023-06-26,TESLA INC,NaN,NaN,240.70000,258.37000,241.05000,179758166.0,-0.060600,241.02000,241.05000,3169504.0,1940000.0
52770749,93436,2023-06-27,TESLA INC,NaN,NaN,240.85001,250.38989,250.21001,164827712.0,0.038000,250.20000,250.21001,3169504.0,1630000.0
52770750,93436,2023-06-28,TESLA INC,NaN,NaN,248.89000,259.88000,256.23999,159593987.0,0.024100,256.39001,256.41000,3169504.0,1550000.0
52770751,93436,2023-06-29,TESLA INC,NaN,NaN,253.61000,260.73999,257.50000,131177205.0,0.004917,257.51999,257.54001,3169504.0,1320000.0


In [82]:
#Change every negative stock price into positive numbers and delete all the rows with dislisted stock and missing return.
daily_stock01['PRC'] = abs(daily_stock['PRC'])
#daily_stock.dropna(axis='index', how='any', subset=['RET'])
daily_stock02 = daily_stock01.dropna(axis='index', how='any', subset=['RET'])

C:\Users\23036531R\AppData\Local\Temp\ipykernel_3748\2444857973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_stock01['PRC'] = abs(daily_stock['PRC'])


In [83]:
daily_stock03 = daily_stock02['DLSTCD']

In [84]:
daily_stock04 = daily_stock02[daily_stock03.isnull().values==True]

In [85]:
daily_stock04.drop("DLSTCD", axis=1, inplace=True)
daily_stock04.drop("DLRET", axis=1, inplace=True)

C:\Users\23036531R\AppData\Local\Temp\ipykernel_3748\3203297701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_stock04.drop("DLSTCD", axis=1, inplace=True)
C:\Users\23036531R\AppData\Local\Temp\ipykernel_3748\3203297701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_stock04.drop("DLRET", axis=1, inplace=True)


In [86]:
daily_stock05 = daily_stock04.reset_index(drop = True)

In [87]:
daily_stock05
daily_stock06 = daily_stock05.drop(labels = 'PERMNO', axis = 1)

In [88]:
daily_stock06

,date,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,NUMTRD
0,1996-01-02,ENERGY WEST INC,8.75000,9.50000,9.12500,0.0,-0.026667,8.75000,9.50000,2281.0,0.0
1,1996-01-03,ENERGY WEST INC,9.50000,9.50000,9.50000,119.0,0.041096,8.75000,9.50000,2281.0,1.0
2,1996-01-04,ENERGY WEST INC,8.75000,9.50000,9.12500,0.0,-0.039474,8.75000,9.50000,2281.0,0.0
3,1996-01-05,ENERGY WEST INC,8.75000,8.75000,8.75000,1384.0,-0.041096,8.75000,9.50000,2281.0,2.0
4,1996-01-08,ENERGY WEST INC,8.75000,9.50000,8.75000,1000.0,0.000000,8.75000,9.50000,2281.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
51981194,2023-06-23,TESLA INC,252.80000,262.45001,256.60001,177346084.0,-0.030271,256.51999,256.54999,3169504.0,1710000.0
51981195,2023-06-26,TESLA INC,240.70000,258.37000,241.05000,179758166.0,-0.060600,241.02000,241.05000,3169504.0,1940000.0
51981196,2023-06-27,TESLA INC,240.85001,250.38989,250.21001,164827712.0,0.038000,250.20000,250.21001,3169504.0,1630000.0
51981197,2023-06-28,TESLA INC,248.89000,259.88000,256.23999,159593987.0,0.024100,256.39001,256.41000,3169504.0,1550000.0


In [89]:
Date = daily_stock06['date']
Date.date=pd.to_datetime(Date)
DATE = Date.date
daily_stock06["day"] = DATE.map(lambda x: x.day)
daily_stock06["month"] = DATE.map(lambda x: x.month)
daily_stock06["year"] = DATE.map(lambda x: x.year)

In [90]:
daily_stock06

,date,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,NUMTRD,day,month,year
0,1996-01-02,ENERGY WEST INC,8.75000,9.50000,9.12500,0.0,-0.026667,8.75000,9.50000,2281.0,0.0,2,1,1996
1,1996-01-03,ENERGY WEST INC,9.50000,9.50000,9.50000,119.0,0.041096,8.75000,9.50000,2281.0,1.0,3,1,1996
2,1996-01-04,ENERGY WEST INC,8.75000,9.50000,9.12500,0.0,-0.039474,8.75000,9.50000,2281.0,0.0,4,1,1996
3,1996-01-05,ENERGY WEST INC,8.75000,8.75000,8.75000,1384.0,-0.041096,8.75000,9.50000,2281.0,2.0,5,1,1996
4,1996-01-08,ENERGY WEST INC,8.75000,9.50000,8.75000,1000.0,0.000000,8.75000,9.50000,2281.0,2.0,8,1,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51981194,2023-06-23,TESLA INC,252.80000,262.45001,256.60001,177346084.0,-0.030271,256.51999,256.54999,3169504.0,1710000.0,23,6,2023
51981195,2023-06-26,TESLA INC,240.70000,258.37000,241.05000,179758166.0,-0.060600,241.02000,241.05000,3169504.0,1940000.0,26,6,2023
51981196,2023-06-27,TESLA INC,240.85001,250.38989,250.21001,164827712.0,0.038000,250.20000,250.21001,3169504.0,1630000.0,27,6,2023
51981197,2023-06-28,TESLA INC,248.89000,259.88000,256.23999,159593987.0,0.024100,256.39001,256.41000,3169504.0,1550000.0,28,6,2023


In [91]:
#What we really need here is the stock return RET, so in order to reduce the file size. I delete all the other useless columns.
daily_stock07 = daily_stock06.drop(labels = ['BIDLO', 'ASKHI', 'PRC', 'VOL', 'BID', 'ASK', 'SHROUT', 'NUMTRD'], axis = 1)

In [92]:
daily_stock07

,date,COMNAM,RET,day,month,year
0,1996-01-02,ENERGY WEST INC,-0.026667,2,1,1996
1,1996-01-03,ENERGY WEST INC,0.041096,3,1,1996
2,1996-01-04,ENERGY WEST INC,-0.039474,4,1,1996
3,1996-01-05,ENERGY WEST INC,-0.041096,5,1,1996
4,1996-01-08,ENERGY WEST INC,0.000000,8,1,1996
...,...,...,...,...,...,...
51981194,2023-06-23,TESLA INC,-0.030271,23,6,2023
51981195,2023-06-26,TESLA INC,-0.060600,26,6,2023
51981196,2023-06-27,TESLA INC,0.038000,27,6,2023
51981197,2023-06-28,TESLA INC,0.024100,28,6,2023


In [95]:
#save daily_stock07 as question5 csv file to prepare for the next stage.
#daily_stock07.to_csv("question5.csv", index=False)